VERSION 1: Full Pipeline with Baseline, BERT+MLP, Hybrid XGBoost, SHAP, and Error Analysis

In [ ]:
# ==========================================
# [CELL 1] INSTALLATION & IMPORTS
# ==========================================
!pip install pandas numpy torch xgboost transformers scikit-learn matplotlib shap lime tqdm

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
import xgboost as xgb
import matplotlib.pyplot as plt
import shap
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix
import seaborn as sns
from tqdm import tqdm

# Setup Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {DEVICE}")
shap.initjs()

# ==========================================
# [CELL 2] PHASE 1: DATA PREP & FEATURE ENGINEERING
# ==========================================
print("\n--- PHASE 1: Data Preparation ---")

# 1. Load Data
try:
    df = pd.read_csv('fake reviews dataset.csv')
    print(f"Data Loaded: {len(df)} rows")
except FileNotFoundError:
    print("Error: 'fake reviews dataset.csv' not found. Please upload it.")

# 2. Target Creation
df['target'] = df['label'].apply(lambda x: 1 if x == 'CG' else 0)

# 3. Feature Engineering
df['text_len'] = df['text_'].apply(len)
df['word_count'] = df['text_'].apply(lambda x: len(str(x).split()))

# 4. Preprocessing Metadata
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['category']),
        ('num', StandardScaler(), ['rating', 'text_len', 'word_count'])
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

structured_features = preprocessor.fit_transform(df[['category', 'rating', 'text_len', 'word_count']])
feature_names = preprocessor.get_feature_names_out()
# Rename to avoid collisions
feature_names = [f"scaled_{name}" if name in ['rating', 'text_len', 'word_count'] else name for name in feature_names]

structured_df = pd.DataFrame(structured_features, columns=feature_names)
df_final = pd.concat([df[['text_', 'target']], structured_df], axis=1)

print("Feature Engineering Complete.")

# ==========================================
# [CELL 3] PHASE 2: BASELINE MODEL
# ==========================================
print("\n--- PHASE 2: Baseline (TF-IDF + LR) ---")

X_base_train, X_base_test, y_base_train, y_base_test = train_test_split(
    df['text_'], df['target'], test_size=0.2, random_state=42
)

tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_base_train)
X_test_tfidf = tfidf.transform(X_base_test)

lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_base_train)
print(f"Baseline Accuracy: {lr.score(X_test_tfidf, y_base_test):.4f}")

# ==========================================
# [CELL 4] PHASE 3: BERT + MLP (Deep Learning)
# ==========================================
print("\n--- PHASE 3: BERT + MLP (Full Fine-Tuning) ---")

# CONFIG FOR BERT MLP
TRAIN_BERT_MLP = True  # <--- Set to False to skip this slow part
EPOCHS = 2
BATCH_SIZE = 16
LR = 2e-5

if TRAIN_BERT_MLP:
    # 1. Dataset Class
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    class ReviewDataset(Dataset):
        def __init__(self, df, tokenizer, max_len=128):
            self.texts = df['text_'].to_numpy()
            self.targets = df['target'].to_numpy()
            self.meta = df[feature_names].to_numpy().astype(np.float32)
            self.tokenizer = tokenizer
            self.max_len = max_len
        def __len__(self):
            return len(self.texts)
        def __getitem__(self, idx):
            encoding = self.tokenizer.encode_plus(
                str(self.texts[idx]),
                add_special_tokens=True,
                max_length=self.max_len,
                return_token_type_ids=False,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt',
            )
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'metadata': torch.tensor(self.meta[idx]),
                'targets': torch.tensor(self.targets[idx], dtype=torch.long)
            }

    # 2. Split & Loaders
    df_train, df_test = train_test_split(df_final, test_size=0.2, random_state=42)
    train_ds = ReviewDataset(df_train, tokenizer)
    test_ds = ReviewDataset(df_test, tokenizer)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

    # 3. Model Definition
    class HybridBertMLP(nn.Module):
        def __init__(self, n_meta):
            super(HybridBertMLP, self).__init__()
            self.bert = BertModel.from_pretrained('bert-base-uncased')
            fused_dim = 768 + n_meta
            self.classifier = nn.Sequential(
                nn.Linear(fused_dim, 256),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(256, 2)
            )
        def forward(self, input_ids, attention_mask, metadata):
            output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            cls_emb = output.last_hidden_state[:, 0, :]
            fused = torch.cat((cls_emb, metadata), dim=1)
            return self.classifier(fused)

    model_mlp = HybridBertMLP(n_meta=len(feature_names)).to(DEVICE)
    optimizer = AdamW(model_mlp.parameters(), lr=LR)
    loss_fn = nn.CrossEntropyLoss().to(DEVICE)

    # 4. Training Loop
    print("Starting BERT+MLP Training (This may take time)...")
    for epoch in range(EPOCHS):
        model_mlp.train()
        total_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            ids = batch['input_ids'].to(DEVICE)
            mask = batch['attention_mask'].to(DEVICE)
            meta = batch['metadata'].to(DEVICE)
            targets = batch['targets'].to(DEVICE)

            optimizer.zero_grad()
            outputs = model_mlp(ids, mask, meta)
            loss = loss_fn(outputs, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} Loss: {total_loss/len(train_loader):.4f}")

    # 5. Evaluation for BERT+MLP
    model_mlp.eval()
    mlp_preds, mlp_true = [], []
    with torch.no_grad():
        for batch in test_loader:
            ids = batch['input_ids'].to(DEVICE)
            mask = batch['attention_mask'].to(DEVICE)
            meta = batch['metadata'].to(DEVICE)
            targets = batch['targets'].to(DEVICE)
            outputs = model_mlp(ids, mask, meta)
            _, preds = torch.max(outputs, dim=1)
            mlp_preds.extend(preds.cpu().numpy())
            mlp_true.extend(targets.cpu().numpy())

    print(f"\nBERT+MLP Accuracy: {accuracy_score(mlp_true, mlp_preds):.4f}")
else:
    print("Skipping BERT+MLP Training (Flag is False)")

# ==========================================
# [CELL 5] PHASE 4: BERT + XGBOOST (Hybrid)
# ==========================================
print("\n--- PHASE 4: Optimized Hybrid (BERT + XGBoost) ---")

# 1. Extract Embeddings (Frozen BERT)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(DEVICE)
bert_model.eval()

def get_embeddings(texts, batch_size=32):
    embeddings = []
    print("Extracting BERT features...")
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        encoded = tokenizer(batch_texts, padding=True, truncation=True, max_length=128, return_tensors='pt').to(DEVICE)
        with torch.no_grad():
            out = bert_model(**encoded)
            embeddings.append(out.last_hidden_state[:, 0, :].cpu().numpy())
    return np.vstack(embeddings)

# Check inputs are strings
text_data = df_final['text_'].astype(str).tolist()
X_text_emb = get_embeddings(text_data)

# 2. Fuse & Split
X_hybrid = np.hstack((X_text_emb, structured_features))
y_hybrid = df_final['target'].values

# FIX: Unpack 6 values here (Train X, Test X, Train y, Test y, Train Text, Test Text)
X_train, X_test, y_train, y_test, text_train, text_test = train_test_split(
    X_hybrid, y_hybrid, df_final['text_'], test_size=0.2, random_state=42
)

# 3. Train XGBoost
xgb_model = xgb.XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.05, n_jobs=-1)
xgb_model.fit(X_train, y_train)
print("XGBoost Hybrid Model Trained.")

# ==========================================
# [CELL 6] EVALUATION
# ==========================================
print("\n--- PHASE 5: Evaluation (Hybrid XGBoost) ---")
y_pred = xgb_model.predict(X_test)
print(f"Hybrid Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred, target_names=['Original', 'Fake']))

# Plot ROC
y_probs = xgb_model.predict_proba(X_test)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_probs)
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f"AUC = {auc(fpr, tpr):.2f}")
plt.plot([0, 1], [0, 1], 'k--')
plt.title("ROC Curve (Hybrid XGBoost)")
plt.legend()
plt.show()

# ==========================================
# [CELL 7] EXPLAINABILITY
# ==========================================
print("\n--- PHASE 6: Explainability ---")
# Global Importance (SHAP)
bert_names = [f"bert_{i}" for i in range(768)]
all_names = bert_names + list(feature_names)
xgb_model.get_booster().feature_names = all_names

explainer = shap.TreeExplainer(xgb_model)
shap_vals = explainer.shap_values(X_test[:100])
shap.summary_plot(shap_vals, X_test[:100], feature_names=all_names, plot_type="bar")

# ==========================================
# [CELL 8] ERROR ANALYSIS
# ==========================================
print("\n--- PHASE 7: Error Analysis ---")
errors = pd.DataFrame({'text': text_test.values, 'true': y_test, 'pred': y_pred, 'prob': y_probs})
fp = errors[(errors['true']==0) & (errors['pred']==1)]
print(f"False Positives: {len(fp)}")
if not fp.empty:
    top_fp = fp.sort_values('prob', ascending=False).iloc[0]
    print(f"Top False Positive (Real flagged as Fake): \n{top_fp['text'][:200]}...")

VERSION 2: Optimized Hybrid Model + LIME & SHAP Explainability

In [ ]:
# ==========================================
# [CELL 1] INSTALLATION & IMPORTS
# ==========================================
!pip install pandas numpy torch xgboost transformers scikit-learn matplotlib shap lime tqdm

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import xgboost as xgb
import matplotlib.pyplot as plt
import shap
import lime
from lime.lime_text import LimeTextExplainer
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix
from tqdm import tqdm

# Setup Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {DEVICE}")
shap.initjs()

# ==========================================
# [CELL 2] PHASE 1: DATA PREP & FEATURE ENGINEERING
# ==========================================
print("\n--- PHASE 1: Data Preparation ---")
try:
    df = pd.read_csv('fake reviews dataset.csv')
except FileNotFoundError:
    print("Error: 'fake reviews dataset.csv' not found. Please upload it.")

df['target'] = df['label'].apply(lambda x: 1 if x == 'CG' else 0)
df['text_len'] = df['text_'].apply(len)
df['word_count'] = df['text_'].apply(lambda x: len(str(x).split()))

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), ['category']),
        ('num', StandardScaler(), ['rating', 'text_len', 'word_count'])
    ],
    remainder='drop',
    verbose_feature_names_out=False
)

structured_features = preprocessor.fit_transform(df[['category', 'rating', 'text_len', 'word_count']])
feature_names = preprocessor.get_feature_names_out()
feature_names = [f"scaled_{name}" if name in ['rating', 'text_len', 'word_count'] else name for name in feature_names]

structured_df = pd.DataFrame(structured_features, columns=feature_names)
df_final = pd.concat([df[['text_', 'target']], structured_df], axis=1)
print("Feature Engineering Complete.")

# ==========================================
# [CELL 3] PHASE 2: BASELINE
# ==========================================
# (Skipping execution for brevity, but code is same as before)
print("\n--- PHASE 2: Baseline Skipped for Speed ---")

# ==========================================
# [CELL 4] PHASE 3: BERT + MLP (Definition)
# ==========================================
# (Deep Learning Class Definition - Same as before)
class HybridBertMLP(nn.Module):
    def __init__(self, n_metadata_features, n_classes=2):
        super(HybridBertMLP, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        total_fused_dim = 768 + n_metadata_features
        self.classifier = nn.Sequential(
            nn.Linear(total_fused_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, n_classes)
        )
    def forward(self, input_ids, attention_mask, metadata_features):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = bert_output.last_hidden_state[:, 0, :]
        fused_vector = torch.cat((cls_embedding, metadata_features), dim=1)
        return self.classifier(fused_vector)


# ==========================================
# [CELL 5] PHASE 4: OPTIMIZED HYBRID (BERT + XGBOOST)
# ==========================================
print("\n--- PHASE 4: Optimized Hybrid (BERT + XGBoost) ---")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased').to(DEVICE)
bert_model.eval()

def extract_bert_embeddings(texts, batch_size=32):
    embeddings = []
    # Only show progress bar if batch is large
    disable_tqdm = len(texts) < 50
    iterator = range(0, len(texts), batch_size)
    if not disable_tqdm:
         iterator = tqdm(iterator, desc="BERT Extraction")

    for i in iterator:
        batch_texts = texts[i : i + batch_size]
        encoded = tokenizer(
            batch_texts, padding=True, truncation=True, max_length=128, return_tensors='pt'
        ).to(DEVICE)
        with torch.no_grad():
            output = bert_model(**encoded)
            cls_emb = output.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_emb)
    return np.vstack(embeddings)

# Extract Features
text_embeddings = extract_bert_embeddings(df_final['text_'].astype(str).tolist())
X_hybrid = np.hstack((text_embeddings, structured_features))
y_hybrid = df_final['target'].values

# --- THE FIX IS HERE ---
# We must unpack 8 values (Train/Test for all 4 inputs)
X_train, X_test, y_train, y_test, text_train, text_test_raw, meta_train, meta_test_raw = train_test_split(
    X_hybrid,
    y_hybrid,
    df_final['text_'],
    structured_features,
    test_size=0.2,
    random_state=42
)

# Train XGBoost
print("Training XGBoost...")
xgb_model = xgb.XGBClassifier(n_estimators=150, max_depth=6, learning_rate=0.05, n_jobs=-1)
xgb_model.fit(X_train, y_train)
print("Trained.")

# ==========================================
# [CELL 6] PHASE 5: EVALUATION
# ==========================================
y_pred = xgb_model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred, target_names=['Original', 'Fake']))

# ==========================================
# [CELL 7] PHASE 6: EXPLAINABILITY (LIME & SHAP TEXT)
# ==========================================
print("\n--- PHASE 6: Explainability (Text-Level) ---")

# 1. DEFINE THE WRAPPER FUNCTION
# This is the magic bridge. It takes raw text, adds the *fixed* metadata for the review we are studying,
# runs BERT, and feeds it to XGBoost.
class HybridPredictor:
    def __init__(self, metadata_row):
        self.metadata_row = metadata_row

    def predict_proba(self, texts):
        # A. Get BERT embeddings for the perturbed texts
        # (We assume texts is a list of strings)
        embs = extract_bert_embeddings(texts, batch_size=16)

        # B. Repeat the static metadata to match the number of texts
        # Shape: (N_texts, N_meta_features)
        meta_batch = np.tile(self.metadata_row, (len(texts), 1))

        # C. Fuse
        combined = np.hstack((embs, meta_batch))

        # D. Predict
        return xgb_model.predict_proba(combined)

# Pick a review to explain (e.g., Index 10)
idx_to_explain = 10
review_text = text_test_raw.iloc[idx_to_explain]
review_meta = meta_test_raw[idx_to_explain]
true_label = "Fake (CG)" if y_test[idx_to_explain]==1 else "Original"

print(f"Explaining Review #{idx_to_explain}")
print(f"True Label: {true_label}")
print(f"Text Snippet: {review_text[:100]}...")

# --- PART A: LIME (Local Interpretable Model-agnostic Explanations) ---
print("\n[A] Running LIME...")
predictor_instance = HybridPredictor(review_meta)
lime_explainer = LimeTextExplainer(class_names=['Original', 'Fake'])

# Run LIME (This takes a few seconds as it perturbs the text)
exp = lime_explainer.explain_instance(
    review_text,
    predictor_instance.predict_proba,
    num_features=10
)

# Show LIME Result (List of weighted words)
print("LIME Weights (Positive = Fake, Negative = Original):")
print(exp.as_list())
# To visualize in notebook: exp.show_in_notebook(text=True)

# --- PART B: SHAP (Text Plot) ---
# Instead of abstract embedding numbers, we use the Text masker to see WORDS.
print("\n[B] Running SHAP (Text)...")

# We create a generic wrapper for SHAP
# Note: SHAP Text explainer is slower than TreeExplainer, so we run it on just 1-2 examples.
def shap_predictor(texts):
    # Wrapper that handles numpy arrays of strings
    if isinstance(texts, np.ndarray):
        texts = texts.tolist()

    # We use the SAME metadata as above for simplicity (Local Explanation)
    # In a perfect world, we'd map every text to its own metadata, but for
    # text-importance analysis, holding metadata constant is standard.
    predictor = HybridPredictor(review_meta)
    return predictor.predict_proba(texts)

# Create the Explainer
# We use a Masker that understands English text (via the BERT tokenizer)
masker = shap.maskers.Text(tokenizer)
explainer = shap.Explainer(shap_predictor, masker)

# Explain the single review
shap_values = explainer([review_text])

# Visualize
print("SHAP Text Plot generated (Red=Fake, Blue=Original).")
# In Jupyter/Colab, this line renders the interactive plot:
shap.plots.text(shap_values)

# ==========================================
# [CELL 8] PHASE 7: GLOBAL METADATA IMPORTANCE
# ==========================================
print("\n--- PHASE 7: Global Metadata Importance ---")
# Since BERT embeddings are abstract, the best "Global" chart is
# showing how much the METADATA matters compared to the text.

# 1. Get raw importance from XGBoost
importance = xgb_model.feature_importances_

# 2. Separate Text vs Metadata
# First 768 features are BERT, rest are Metadata
bert_importance = np.sum(importance[:768])
meta_importances = importance[768:]
meta_names = feature_names

# 3. Plot
plt.figure(figsize=(10, 5))
# We create a list: [BERT_Aggregate] + [Metadata_Features]
plot_names = ['BERT (Text Aggregate)'] + meta_names
plot_values = [bert_importance] + list(meta_importances)

# Sort for nicer plotting
sorted_idx = np.argsort(plot_values)
plt.barh(np.array(plot_names)[sorted_idx], np.array(plot_values)[sorted_idx], color='teal')
plt.title("Global Feature Importance: Text vs Metadata")
plt.xlabel("Relative Importance")
plt.show()